<a href="https://colab.research.google.com/github/cruz-marco/dex_files/blob/main/SAFE_Marco_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.0 Importando Libs

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.metrics.pairwise import cosine_similarity

## 0.1 acessando dataset no Kaggle

# 1.0 Recebendo os dados

In [ ]:
DATA_PATH = Path('/content/book-recommendation-dataset')

#montando os dataframes
books, ratings, users = (pd.read_csv(DATA_PATH / 'Books.csv') \
                         .drop(['Image-URL-S', 'Image-URL-M',
                                'Image-URL-L'],axis=1), 
                         pd.read_csv(DATA_PATH / 'Ratings.csv'), 
                         pd.read_csv(DATA_PATH / 'Users.csv'))

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
for frame, desc in zip(('books', 'ratings', 'users'),
                      ('Livros', 'Avaliações', 'Usuários')):
  display(
      eval(frame) \
      .head(3) \
      .style.set_caption(desc)
  )


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0


,User-ID,Location,Age
0,1,"nyc, new york, usa",nan
1,2,"stockton, california, usa",18.000000
2,3,"moscow, yukon territory, russia",nan


# 2.0 Recomendações

## 2.1 Recomendações mais bem avaliadas

In [ ]:
t10_eval_bks = ratings[ratings['Book-Rating'] > 0] \
                .groupby(['ISBN']) \
                .agg({'Book-Rating': 'sum'}) \
                .reset_index() \
                .merge(books, how='right', on='ISBN') \
                .sort_values('Book-Rating', ascending = False) \
                .head(10)

In [ ]:
display(t10_eval_bks)

,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
408,0316666343,5787.0,The Lovely Bones: A Novel,Alice Sebold,2002,"Little, Brown"
748,0385504209,4108.0,The Da Vinci Code,Dan Brown,2003,Doubleday
522,0312195516,3134.0,The Red Tent (Bestselling Backlist),Anita Diamant,1998,Picador USA
2143,059035342X,2798.0,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,1999,Arthur A. Levine Books
356,0142001740,2595.0,The Secret Life of Bees,Sue Monk Kidd,2003,Penguin Books
26,0971880107,2551.0,Wild Animus,Rich Shapero,2004,Too Far
1105,0060928336,2524.0,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Rebecca Wells,1997,Perennial
706,0446672211,2402.0,Where the Heart Is (Oprah's Book Club (Paperba...,Billie Letts,1998,Warner Books
231,0452282152,2219.0,Girl with a Pearl Earring,Tracy Chevalier,2001,Plume Books
118,0671027360,2179.0,Angels &amp; Demons,Dan Brown,2001,Pocket Star


## 2.2 Recomendação de items mais populares.

In [ ]:
t10_pop_bks = ratings.groupby(['ISBN']) \
              .agg({'User-ID': 'count'}) \
              .reset_index() \
              .rename(columns={'User-ID' : 'User-count'}) \
              .merge(books, how='right', on='ISBN') \
              .sort_values('User-count', ascending = False) \
              .head(10)

In [ ]:
display(t10_pop_bks)

,ISBN,User-count,Book-Title,Book-Author,Year-Of-Publication,Publisher
26,0971880107,2502.0,Wild Animus,Rich Shapero,2004,Too Far
408,0316666343,1295.0,The Lovely Bones: A Novel,Alice Sebold,2002,"Little, Brown"
748,0385504209,883.0,The Da Vinci Code,Dan Brown,2003,Doubleday
1105,0060928336,732.0,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Rebecca Wells,1997,Perennial
522,0312195516,723.0,The Red Tent (Bestselling Backlist),Anita Diamant,1998,Picador USA
305,044023722X,647.0,A Painted House,John Grisham,2001,Dell Publishing Company
356,0142001740,615.0,The Secret Life of Bees,Sue Monk Kidd,2003,Penguin Books
1922,067976402X,614.0,Snow Falling on Cedars,David Guterson,1995,Vintage Books USA
118,0671027360,586.0,Angels &amp; Demons,Dan Brown,2001,Pocket Star
706,0446672211,585.0,Where the Heart Is (Oprah's Book Club (Paperba...,Billie Letts,1998,Warner Books


# 3.0 Criando a matriz de relacionamento

In [ ]:
ratings.shape, ratings['User-ID'].nunique(), ratings['ISBN'].nunique()

((1149780, 3), 105283, 340556)

In [ ]:
t10_usrs = ratings.groupby('User-ID') \
          .agg({'ISBN': 'count'}) \
          .rename(columns={'ISBN' : 'ISBN_count'}) \
          .sort_values('ISBN_count', ascending = False) \
          .head(1000)

In [ ]:
rel_matrix = ratings[ratings['User-ID'].isin(t10_usrs.index)]\
            .rename(columns={
                'User-ID' : 'usr',
                'ISBN' : 'bks',
                'Book-Rating' : 'act'
            })\
            .pivot_table(index='usr', columns='bks', values='act')\
            .fillna(0)

In [ ]:
rel_matrix

bks,0330299891,0375404120,0586045007,9022906116,9032803328,9044922564,9044922572,9044922718,9044923161,904492401X,...,UNGRANDHOMMED,X000000000,"YOUTELLEM,AND",ZR903CX0003,"\0432534220\""""","\2842053052\""""",b00005wz75,cn108465,cn113107,Ô½crosoft
usr,,,,,,,,,,,,,,,,,,,,,
254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2766,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277427,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
277478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
277639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 4.0 Concentração de consumo

In [ ]:
def get_data(df, dtype='usr'):
  if dtype == 'usr':
    ret = df.groupby('User-ID')\
          .agg({'ISBN' : 'count'})\
          .rename(columns={'ISBN' : 'ISBN_count'})\
          .sort_values('ISBN_count', ascending = False)
  else:
    ret = df.groupby('ISBN')\
          .agg({'User-ID' : 'count'})\
          .sort_values('User-ID', ascending = False)\
          .rename(columns={'User-ID' : 'User-ID_count'})
  return ret

def get_top(df, size):
  return df.head(size).index.tolist()


## 4.1 User Based

In [ ]:
cos_usr = pd.DataFrame(cosine_similarity(rel_matrix), index=rel_matrix.index,
                       columns=rel_matrix.index)

In [ ]:
cos_usr

usr,254,2033,2276,2766,2977,3363,3757,4017,4385,6242,...,274301,274308,274808,275970,276680,277427,277478,277639,278188,278418
usr,,,,,,,,,,,,,,,,,,,,,
254,1.000000,0.052825,0.000000,0.000000,0.016767,0.013230,0.0,0.010541,0.000000,0.000000,...,0.000000,0.022323,0.0,0.012727,0.016365,0.000000,0.000000,0.000000,0.0,0.000000
2033,0.052825,1.000000,0.046753,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2276,0.000000,0.046753,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.005661,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.023342,0.0,0.000000
2766,0.000000,0.000000,0.000000,1.000000,0.000000,0.007233,0.0,0.027081,0.000000,0.006718,...,0.015312,0.000000,0.0,0.006088,0.023016,0.020162,0.015155,0.000000,0.0,0.006667
2977,0.016767,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.007082,0.000000,0.008321,...,0.011092,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277427,0.000000,0.000000,0.000000,0.020162,0.000000,0.000000,0.0,0.007570,0.000000,0.007059,...,0.014820,0.036513,0.0,0.006398,0.029510,1.000000,0.000000,0.000000,0.0,0.000000
277478,0.000000,0.000000,0.000000,0.015155,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.000000
277639,0.000000,0.000000,0.023342,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.000000


## 4.2 Item Based

In [ ]:
top_bks = ratings.pipe(get_data, dtype='bks').pipe(get_top, size=999)

In [ ]:
bks_rel = ratings[ratings['ISBN'].isin(top_bks)]\
          .rename(columns={
              'User-ID' : 'usr',
              'ISBN' : 'bks',
              'Book-Rating' : 'act'
          })\
          .pivot_table(index='usr', columns='bks', values='act')\
          .fillna(0)

In [ ]:
cos_bks = pd.DataFrame(cosine_similarity(bks_rel.T), index=bks_rel.columns,
                       columns=bks_rel.columns)

In [ ]:
cos_bks

bks,000649840X,002542730X,0060008032,0060096195,006016848X,0060173289,0060175400,0060188731,006019491X,0060199652,...,1573225517,1573225789,1573227331,1573229326,1573229571,1573229725,1576737330,1592400876,1878424319,8873122933
bks,,,,,,,,,,,,,,,,,,,,,
000649840X,1.000000,0.059837,0.028577,0.000000,0.043903,0.0,0.020793,0.017269,0.030229,0.024108,...,0.025743,0.036977,0.013844,0.015679,0.018125,0.034746,0.000000,0.022026,0.015513,0.000000
002542730X,0.059837,1.000000,0.015558,0.000000,0.014222,0.0,0.024904,0.009401,0.016457,0.013124,...,0.029431,0.030372,0.039190,0.038208,0.026314,0.000000,0.029855,0.026203,0.000000,0.000000
0060008032,0.028577,0.015558,1.000000,0.036488,0.063930,0.0,0.024103,0.020018,0.056502,0.027945,...,0.000000,0.022559,0.016047,0.036349,0.075038,0.000000,0.000000,0.048228,0.000000,0.000000
0060096195,0.000000,0.000000,0.036488,1.000000,0.000000,0.0,0.015929,0.000000,0.000000,0.000000,...,0.000000,0.011182,0.000000,0.000000,0.000000,0.000000,0.000000,0.027342,0.000000,0.000000
006016848X,0.043903,0.014222,0.063930,0.000000,1.000000,0.0,0.022034,0.018300,0.032032,0.025546,...,0.000000,0.020623,0.014670,0.016614,0.046645,0.000000,0.000000,0.023341,0.033698,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1573229725,0.034746,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.032452,0.000000,0.000000,...,0.065309,0.021943,0.021853,0.032516,0.000000,1.000000,0.040131,0.058639,0.000000,0.000000
1576737330,0.000000,0.029855,0.000000,0.000000,0.000000,0.0,0.031487,0.000000,0.008146,0.024750,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.040131,1.000000,0.025126,0.000000,0.000000
1592400876,0.022026,0.026203,0.048228,0.027342,0.023341,0.0,0.018578,0.033430,0.027007,0.021539,...,0.020445,0.049073,0.012368,0.030018,0.028532,0.058639,0.025126,1.000000,0.000000,0.000000
